In [1]:
import os  
import glob 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import exiftool
import numpy
import re

## ECOWIND-ACCELERATE
# project='ECOWind-accelerate'
# drive ='e:'
# indir = os.path.join(drive,project,'master','files','images')
# save_folder=os.path.join(drive,project,'image-metadata')

## mil
drive ='x:'
indir = os.path.join(drive,'SONAR ARCHIVE','AUV','Archived','SIFIDS','lewis-test')
save_folder=os.path.join(drive,'SONAR ARCHIVE','AUV','Archived','SIFIDS','lewis-test','image-metadata')


folders=glob.glob(indir+'\*')

for folder in folders:
    # get timestamp of picture folder
    splitname=os.path.split(folder)
    tstamp=splitname[1]
    # build filenam for saving
    savename=os.path.join(save_folder,tstamp+'-coords.csv')
    # check for existence of filename
    if not os.path.exists(savename):
    
        files=glob.glob(folder+'\*.jpg')

        df = pd.DataFrame(columns=['Image_Name','path', 'time','capture time','altitude', 'depth','heading','Lat','latDec','Lon','lonDec','pitch','roll','surge','sway'])

        loopy=range(len(files))

        with exiftool.ExifToolHelper() as et:
                metadata = et.get_metadata(files)

        for i in loopy:
            file=files[i]
            files1 = [file]

            comment=metadata[i]['File:Comment']
            time = re.search('time="(.*)">', comment).group(1)
            capture_time = re.search('<capture_time>(.*)</capture_time>', comment).group(1)
            altitude = re.search('<altitude>(.*)</altitude>', comment).group(1)
            depth = re.search('<depth>(.*)</depth>', comment).group(1)
            heading = re.search('<heading>(.*)</heading>', comment).group(1)
            lat = re.search('<lat>(.*)</lat>', comment).group(1)
            lon = re.search('<lon>(.*)</lon>', comment).group(1)
            pitch = re.search('<pitch>(.*)</pitch>', comment).group(1)
            roll = re.search('<roll>(.*)</roll>', comment).group(1)
            surge = re.search('<surge>(.*)</surge>', comment).group(1)
            sway = re.search('<sway>(.*)</sway>', comment).group(1)


            signlat = 1
            if lat[-1] == "S":
                signlat = -1    
            lenlat = len(lat)
            latCor = signlat * (float(lat[:2]) + float(lat[2:lenlat-2])/60.0)

            signlon=1
            if lon[-1] == "W":
                signlon = -1
            lenlon = len(lon)
            lonCor = signlon * (float(lon[:3]) + float(lon[3:lenlon-2])/60.0)


            basename=os.path.basename(file)
            df.loc[i] = [os.path.basename(file),file,time, capture_time,altitude,depth,heading,lat,latCor,lon,lonCor,pitch,roll,surge,sway]


        df.to_csv(os.path.join(save_folder,tstamp+'-coords.csv'))
        
    else:
        print('file ' +savename +' exists already')

KeyError: 'File:Comment'

In [2]:
metadata

[{'SourceFile': 'x:SONAR ARCHIVE/AUV/Archived/SIFIDS/lewis-test/test/frame000001_1499264383_586182.jpg',
  'ExifTool:ExifToolVersion': 12.55,
  'File:FileName': 'frame000001_1499264383_586182.jpg',
  'File:Directory': 'x:SONAR ARCHIVE/AUV/Archived/SIFIDS/lewis-test/test',
  'File:FileSize': 723905,
  'File:FileModifyDate': '2017:07:13 12:04:28+01:00',
  'File:FileAccessDate': '2024:04:10 10:03:01+01:00',
  'File:FileCreateDate': '2024:04:10 10:02:58+01:00',
  'File:FilePermissions': 100666,
  'File:FileType': 'JPEG',
  'File:FileTypeExtension': 'JPG',
  'File:MIMEType': 'image/jpeg',
  'File:ExifByteOrder': 'MM',
  'File:ImageWidth': 1280,
  'File:ImageHeight': 960,
  'File:EncodingProcess': 0,
  'File:BitsPerSample': 8,
  'File:ColorComponents': 1,
  'EXIF:Orientation': 1,
  'EXIF:XResolution': 72,
  'EXIF:YResolution': 72,
  'EXIF:ResolutionUnit': 2,
  'EXIF:Software': 'Adobe Photoshop CS6 (Windows)',
  'EXIF:ModifyDate': '2017:07:13 12:04:28',
  'EXIF:ColorSpace': 65535,
  'EXIF:Exi

In [ ]:
## count the number of photos
picfolder=os.path.join(drive,'Projects','ECOWind','ECOWind-ACCELERATE','image-metadata')

folders=glob.glob(picfolder+'\*')

pics=[]

for folder in folders:
    df=pd.read_csv(folder)
    npics=len(df.index)
    pics.append(npics)

print('Number of photos is ' + str(np.sum(pics)))

In [ ]:
## concatentate the metadata folders
picfolder=os.path.join(drive,'Projects','ECOWind','ECOWind-ACCELERATE','image-metadata')

folders=glob.glob(picfolder+'\*')

for x,folder in enumerate(folders):
    if x==1:
        df=pd.read_csv(folder)
    else:
        ds=pd.read_csv(folder)
        df=pd.concat([df,ds])
        
df.to_csv(os.path.join(save_folder,'photo-coords.csv'))


In [ ]:
df